In [1]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv("data/total.csv")
df

,drifter,time,longitude,latitude
0,0,51.25,129.311005,34.950001
1,0,51.50,129.401993,35.020000
2,0,51.75,129.514008,35.061001
3,0,52.00,129.524002,35.032001
4,0,52.25,129.626007,35.083000
...,...,...,...,...
15840,30,638.00,147.348999,44.167999
15841,30,638.25,147.307007,44.138000
15842,30,638.50,147.250000,44.146999
15843,30,638.75,147.322998,44.201000


In [3]:
# Create a base map with specific tiles (cartodb positron for a clean map)
base_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5, tiles="cartodb positron")

# Define a color palette for each drifter
color_palette = ['blue', 'green', 'red', 'purple', 'orange', 'pink', 'brown', 'gray', 'olive', 'cyan',
                 'lime', 'teal', 'navy', 'maroon', 'gold', 'indigo', 'violet', 'magenta', 'salmon',
                 'coral', 'sienna', 'turquoise', 'lavender', 'darkgreen', 'dodgerblue', 'slategray',
                 'hotpink', 'chocolate', 'mediumseagreen', 'crimson']

# Create an empty dictionary to store drifter names for the legend
drifter_names = {}

# Loop through each drifter and add their path to the base map
for drifter_id in df['drifter'].unique():
    drifter_data = df[df['drifter'] == drifter_id]
    drifter_color = color_palette[drifter_id % len(color_palette)]  # Choose a color from the palette
    
    # Create a feature group for this drifter
    drifter_group = folium.FeatureGroup(name=f'Drifter {drifter_id}')
    
    # Add a polyline to represent the drifter's path
    folium.PolyLine(
        locations=list(zip(drifter_data['latitude'], drifter_data['longitude'])),
        color=drifter_color,
        weight=2.5,
        opacity=1,
        tooltip=f'Drifter {drifter_id}'
    ).add_to(drifter_group)
    
    # Add the drifter group to the base map
    drifter_group.add_to(base_map)

    # Store drifter names for the legend
    drifter_names[f'Drifter {drifter_id}'] = drifter_color

# Add layer control with custom legend
folium.LayerControl(collapsed=False, overlay=True, control=True, name="Legend").add_to(base_map)

# Add custom legend to the map
legend_html = '<div style="position:fixed; top:10px; left:10px; z-index:1000; background-color:white; padding:10px; border-radius:5px;">'
for drifter_name, color in drifter_names.items():
    legend_html += f'<p><i class="fa fa-circle" style="color:{color};"></i> {drifter_name}</p>'
legend_html += '</div>'

base_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
base_map.save('all_drifters_map_with_legend.html')